In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important on local machine
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC

2021-08-25 15:27:51.412107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-25 15:27:51.412135: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
wandb.init(project='chembl', entity='chrisxx')

config = wandb.config
config.n_atoms = 118
config.n_bonds = 4
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.max_nodes = 400
config.max_edges = 600
config.dim_feedforward = 2048
config.lr = 0.0003
config.n_epochs = 10000
config.lr_adjustment_period = 500
config.patience = 5
config.factor = 0.96
config.minimal_lr = 6e-8
config.target_idx = 7
config.batch_size = 200
config.valid_patience = 100
config.valid_minimal_improvement=0.00
config.model_dir = '../../models/chembl/better_transformer/medium/'
config.data_dir = "/mnt/project/chembl/ChEMBL100_noH/"
config.num_workers = 0
config.prefetch_factor = 2
config.persistent_workers = False
config.load_last = False


wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-08-25 15:28:04.084136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-25 15:28:04.084163: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
torch.multiprocessing.get_all_sharing_strategies()

{'file_descriptor', 'file_system'}

In [8]:
path = config.data_dir

In [9]:
class ChEMBL(InMemoryDataset):
    def __init__(self, fname):
        super().__init__()
        self.data, self.slices = torch.load(fname)

class ChEMBL100NoH(Dataset):
    """ChEMBL dataset of molecules and minimal DFS codes."""
    # create data structure that says which id is in which file...
    def __init__(self, path="/home/wendlerc/leq100_timeout60_sanity",
                       graph_pattern="/home/wendlerc/ChEMBL/preprocessedNoHs_split%d.pt", 
                       csv_file="/home/wendlerc/ChEMBL/small_molecules.csv", 
                       idx_max=64,
                       transform=None):
        self.path = path
        self.fnames = glob.glob(path+"CHEMBL*.pkl")
        self.data = []
        self.csv_file = csv_file
        self.path = path
        self.graph_pattern = graph_pattern 
        self.idx_max = idx_max
        self.prepare()
        
        
    def prepare(self):
        codes_all = {}
        for i in tqdm.tqdm(range(self.idx_max)):
            with open(self.path+"/%d/min_dfs_codes_split%d.json"%(i+1, i), 'r') as f:
                codes = json.load(f)
                for key, val in codes.items():
                    codes_all[key] = val
        graph_pattern = self.graph_pattern
        df = pd.read_csv(self.csv_file, delimiter=';', low_memory=False)
        chembl2smiles = {cid:smiles for cid, smiles in zip(df['ChEMBL ID'], df['Smiles'])}
        for i in range(self.idx_max):
            dataset = ChEMBL(graph_pattern%i)
            print(i)
            for data in tqdm.tqdm(dataset):
                if data.name in codes_all:
                    code = codes_all[data.name]
                    d = {'x':data.x.numpy(),
                          'z':data.z.numpy(),
                          'edge_attr': data.edge_attr.numpy(),
                          'edge_index': data.edge_index.numpy(),
                          'name':data.name,
                          'min_dfs_code': np.asarray(code['min_dfs_code']),
                          'min_dfs_index': np.asarray(code['dfs_index']),
                          'smiles': chembl2smiles[data.name]}
                    data_ = Data(x=torch.tensor(d['x']),
                                z=torch.tensor(d['z']),
                                edge_attr=torch.tensor(d['edge_attr']),
                                edge_index=torch.tensor(d['edge_index'], dtype=torch.long),
                                name=d['name'],
                                min_dfs_code=torch.tensor(d['min_dfs_code']),
                                min_dfs_index=torch.tensor(d['min_dfs_index'], dtype=torch.long),
                                smiles=d['smiles'])
                    self.data += [data_]   
            del dataset
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [10]:
def collate_fn(dlist):
    x_batch = [] 
    z_batch = []
    edge_attr_batch = []
    rnd_code_batch = []
    min_code_batch = []
    for d in dlist:
        rnd_code, rnd_index = dfs_code.rnd_dfs_code_from_torch_geometric(d, 
                                                                         d.z.numpy().tolist(), 
                                                                         np.argmax(d.edge_attr.numpy(), axis=1))
        x_batch += [d.x]
        z_batch += [d.z]#[nn.functional.one_hot(d.z, 118)]#118 elements in periodic table
        edge_attr_batch += [d.edge_attr]
        rnd_code_batch += [torch.tensor(rnd_code)]
        min_code_batch += [d.min_dfs_code]
    return rnd_code_batch, x_batch, z_batch, edge_attr_batch, min_code_batch

In [11]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [12]:
dataset = ChEMBL100NoH(idx_max=64)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [02:21<00:00,  2.22s/it]


0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29917/29917 [00:05<00:00, 5408.88it/s]


1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29917/29917 [00:05<00:00, 5235.99it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29916/29916 [00:06<00:00, 4691.68it/s]


62


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29916/29916 [00:05<00:00, 5114.75it/s]


63


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29916/29916 [00:05<00:00, 5161.77it/s]


In [13]:
loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)

In [14]:
to_cuda = lambda T: [t.to(device) for t in T]

In [15]:
model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_encoder=nn.Embedding(config.n_atoms, config.emb_dim), 
                         bond_encoder=nn.Linear(config.n_bonds, config.emb_dim))

In [16]:
if config.load_last:
    model.load_state_dict(torch.load(config.model_dir+'checkpoint.pt'))

In [17]:
optim = optimizers.Adam(model.parameters(), lr=config.lr)

#lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
lr_scheduler = optimizers.lr_scheduler.ExponentialLR(optim, gamma=config.factor)

early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss(ignore_index=-1)
softmax = nn.Softmax(dim=2)

In [18]:
model.to(device)

DFSCodeSeq2SeqFC(
  (encoder): DFSCodeEncoder(
    (emb_dfs): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (emb_seq): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (emb_atom): Embedding(118, 120)
    (emb_bond): Linear(in_features=4, out_features=120, bias=True)
    (mixer): Linear(in_features=600, out_features=600, bias=True)
    (enc): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=600, out_features=600, bias=True)
          )
          (linear1): Linear(in_features=600, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=600, bias=True)
          (norm1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
         

In [ ]:
try:
    for epoch in range(config.n_epochs):  
        epoch_loss = 0
        pbar = tqdm.tqdm(loader)
        for i, data in enumerate(pbar):
            optim.zero_grad()
            rndc, x, z, eattr, minc = data
            rndc = to_cuda(rndc)
            z = to_cuda(z)
            eattr = to_cuda(eattr)
            minc = to_cuda(minc)
            #prepare labels
            minc = [torch.cat((c, (-1)*torch.ones((1, 8), dtype=torch.long, device=device)), dim=0) for c in minc]
            minc_seq = nn.utils.rnn.pad_sequence(minc, padding_value=-1)
            
            #prediction
            dfs1, dfs2, atm1, atm2, bnd, eos = model(rndc, z, eattr)
            eos_label = (minc_seq[:,:,0] == (-1))
            loss = ce(torch.reshape(dfs1, (-1, config.max_nodes)), minc_seq[:, :, 0].view(-1)) 
            loss += ce(torch.reshape(dfs2, (-1, config.max_nodes)), minc_seq[:, :, 1].view(-1))
            loss += ce(torch.reshape(atm1, (-1, config.n_atoms)), minc_seq[:, :, 2].view(-1))
            loss += ce(torch.reshape(bnd, (-1, config.n_bonds)), minc_seq[:, :, 3].view(-1))
            loss += ce(torch.reshape(atm2, (-1, config.n_atoms)), minc_seq[:, :, 4].view(-1))
            loss += bce(eos, torch.unsqueeze(eos_label.float(), -1))
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optim.step()
            epoch_loss = (epoch_loss*i + loss.item())/(i+1)
            
            curr_lr = list(optim.param_groups)[0]['lr']
            wandb.log({'loss':epoch_loss, 
                   'learning rate':curr_lr})
            pbar.set_description('Epoch %d: CE %2.6f'%(epoch+1, epoch_loss))
            
            if i % config.lr_adjustment_period == 0:
                early_stopping(epoch_loss, model)
                
        lr_scheduler.step()#(epoch_loss)
        
        if early_stopping.early_stop:
            break

        if curr_lr < config.minimal_lr:
            break

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


Epoch 3: CE 2.843090:   0%|                                                                                                                                                                               | 1/9541 [00:00<2:17:20,  1.16it/s]

EarlyStopping counter: 1 out of 100


Epoch 4: CE 2.203739:   5%|█████████                                                                                                                                                                    | 501/9541 [05:24<1:38:12,  1.53it/s]

EarlyStopping counter: 1 out of 100


Epoch 4: CE 2.194216:  10%|██████████████████                                                                                                                                                          | 1001/9541 [10:46<1:34:14,  1.51it/s]

...

Epoch 15: CE 1.068073:  47%|█████████████████████████████████████████████████████████████████████████████████▌                                                                                           | 4501/9541 [49:21<53:24,  1.57it/s]

EarlyStopping counter: 2 out of 100


Epoch 15: CE 1.067993:  52%|██████████████████████████████████████████████████████████████████████████████████████████▋                                                                                  | 5001/9541 [54:49<50:32,  1.50it/s]

EarlyStopping counter: 3 out of 100


Epoch 15: CE 1.067410:  57%|██████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                        | 5479/9541 [1:00:06<47:07,  1.44it/s]